# Run simulations for all examples 

Requires ngspice to be in the path.

In [37]:
import json
import circuits as cir
import ngspice
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

def simulate_example_typical(circuit_path):
    #load the cicruit definitions and targets defined in circuit_setup.json
    circuit = cir.Circuit(circuit_path, corners=None)

    #load the defaul example sizing
    with open(circuit_path + "sizing_example.json", 'r') as file:
        sizing = json.load(file)

    return circuit.simulate(sizing)




<ipython-input-37-1f9b34bcaa08>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [40]:
#Folded Cascode OTA PTM130 and SKY 130

print("Folded Cascode SKY 130 TT Simulation Results")
simulation_result = simulate_example_typical("../circuit_examples/folded_cascode_ota/sky130/")

# rescales CL parameter for visialization purposes
df = pd.DataFrame(simulation_result[0]).transpose()
#df['CL'] *= 1e12  
#df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)

Folded Cascode SKY 130 TT Simulation Results
              CL  DELTA_MNM1  DELTA_MNM2  DELTA_MNM3  DELTA_MNM4  DELTA_MNM5  \
TT  6.000000e-12  1.085492   -0.794696   -0.519485   -0.519485   -0.038166     

    DELTA_MNM6  DELTA_MNM7  DELTA_MNM8  DELTA_MPM0  DELTA_MPM1  DELTA_MPM3  \
TT -0.038166   -0.039177   -0.039177    0.996962    0.936917    3.256056     

    DELTA_MPM4  DELTA_MPM5  DELTA_MPM6       GDC       GPS           IDD  \
TT  3.256056   -0.044988   -0.044988   -206.5801 -149.6033  7.247868e-11   

    INOISE_TOTAL  ONOISE_TOTAL  OUTSWING  VOV_MNM1  VOV_MNM2  VOV_MNM3  \
TT  25801.18      0.000008      1.572657  1.041209  1.038469  0.84855    

    VOV_MNM4  VOV_MNM5  VOV_MNM6  VOV_MNM7  VOV_MNM8  VOV_MPM0  VOV_MPM1  \
TT  0.84855  -0.77425  -0.77425  -0.782643 -0.782643  0.362053  0.122726   

    VOV_MPM3  VOV_MPM4  VOV_MPM5  VOV_MPM6  
TT -0.499991 -0.499991 -2.69894  -2.69894   


In [41]:
print("Folded Cascode PTM 130 TT Simulation Results")
simulation_result = simulate_example_typical("../circuit_examples/folded_cascode_ota/ptm130/")

# rescales CL parameter for visialization purposes
df = pd.DataFrame(simulation_result[0]).transpose()
df['CL'] *= 1e12  
df.rename(columns = {"CL": "CL[pF]"}, inplace=True)  
print(df)

Folded Cascode PTM 130 TT Simulation Results
    CL[pF]  DELTA_MNM1  DELTA_MNM2  DELTA_MNM3  DELTA_MNM4  DELTA_MNM5  \
TT  6.0     0.585495    0.739938    0.617565    0.619046    0.413995     

    DELTA_MNM6  DELTA_MNM7  DELTA_MNM8  DELTA_MPM0  DELTA_MPM1  DELTA_MPM3  \
TT  0.639173    0.402403    0.402872    0.472546    0.410293    0.371069     

    DELTA_MPM4  DELTA_MPM5  DELTA_MPM6         FOM         GBW       GDC  \
TT  0.369587    0.534973    0.309809    636.744965  31793610.0  88.39222   

         GPS     IDD  INOISE_TOTAL  ONOISE_TOTAL  OUTSWING       PM  \
TT -3.418366  0.0003  0.000819      0.143896      1.510087  57.6809   

    PM_NEGATIVE      PSRR      SDINOISE      SDONOISE  VOV_MNM1  VOV_MNM2  \
TT -122.3191     91.81059  7.395197e-09  7.966814e-09  0.528085  0.528085   

    VOV_MNM3  VOV_MNM4  VOV_MNM5  VOV_MNM6  VOV_MNM7  VOV_MNM8  VOV_MPM0  \
TT -0.003793 -0.003793  0.001432  0.000963 -0.016224 -0.016224  0.37476    

    VOV_MPM1  VOV_MPM3  VOV_MPM4  VOV_MPM5  V

In [3]:
from scipy.io import loadmat

annots = loadmat('../217m (14).mat')

In [16]:
print(annots['val'][0])
print(len(annots['val'][0]))

t = 0
dt = 0.0027778
with open("data1.txt", "w") as file: 
    for a in annots['val'][0]:
        file.write("{}\t{}\n".format(t, (a*1e-3*0.005)))
        t = t + dt;








[1138 1143 1150 ... 1013 1015 1019]
3600


## Optimizing in Nominal conditions

Outputs are save to the output_folder

This can take a while.


In [3]:
import random
from datetime import datetime
import numpy as np
import pandas as pd
import os


seed = 42
np.random.seed(seed)
random.seed(seed)
nsga2 = opt.NSGA2()

output_folder = "./run/ptm130_folded_cascode/"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)


i = 0
folded_cascode = cir.Circuit("./circuit_examples/ptm130_folded_cascode/", corners=None)
for pop, pop_obj,  pop_cstr, pop_data, evals, front_no in nsga2.minimize(
        folded_cascode,pop_size=256, evaluations=256*100, mutation=0.1, crossover=0.6):
        
        print(i, pop_cstr[pop_cstr.argmax()], datetime.now().time())
        with open("{}history256_{}_{}.json".format(output_folder,seed, i), "w") as file: 
            json.dump(
            {
                "pop":json.loads(pd.DataFrame(data=pop, columns=folded_cascode.parameters).to_json(orient="records")),
                "obj":json.loads(pd.DataFrame(data=pop_obj, columns=folded_cascode.objectives).to_json(orient="records")),
                "cstr":pop_cstr.tolist(),
                "data":pop_data,
                "evals":evals,
                "fn":[ str(fn) for fn in front_no.tolist()]
            },
            file  
            )
        i = i + 1


0 [-2.46806784] 23:15:19.318959
1 [-1.08975836] 23:15:36.184764
2 [-1.08975836] 23:15:54.950172
3 [-1.08975836] 23:16:17.202781
4 [-1.08975836] 23:16:40.593828
5 [-0.42640278] 23:17:06.695972
6 [-0.42640278] 23:17:28.089334
7 [-0.38012679] 23:17:49.446514
8 [-0.30998951] 23:18:15.087804
9 [-0.27559957] 23:18:38.393919
10 [-0.27559957] 23:19:00.872626
11 [-0.27559957] 23:19:20.334230
12 [-0.22000996] 23:19:39.683464
13 [-0.22000996] 23:19:59.836749
14 [-0.22000996] 23:20:21.941143
15 [-0.22000996] 23:20:47.316056
16 [-0.21890357] 23:21:06.089005
17 [-0.13215336] 23:21:24.560100
18 [-0.07368782] 23:21:41.311814
19 [-0.0722602] 23:22:03.381337
20 [-0.05250202] 23:22:27.372538
21 [-0.02386589] 23:22:53.326756
22 [-0.02386589] 23:23:14.525054
23 [-0.00840327] 23:23:39.935825
24 [-0.0018278] 23:24:02.799262
25 [0.] 23:24:21.740788
26 [0.] 23:24:39.644758
27 [0.] 23:25:10.467506
28 [0.] 23:25:29.692839
29 [0.] 23:25:46.394140
30 [0.] 23:26:04.686930
31 [0.] 23:26:34.100390
32 [0.] 23:26:51.07

KeyboardInterrupt: 

In [ ]:
import plot_utils 
importlib.reload(plot_utils)

plot_utils.plt_pof(["{}history256_{}_{}.json".format(output_folder,seed, 100)], scales = [("[uA]", 1e6),("[MHz]", 1e-6),("[dB]", 1.0)])



In [1]:
import os
os.environ


environ{'SHELL': '/bin/bash',
        'VSCODE_VERBOSE_LOGGING': 'true',
        'LIBGL_ALWAYS_INDIRECT': '1',
        'PYTHONUNBUFFERED': '1',
        'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': 'true',
        'VSCODE_LOG_STACK': 'false',
        'WSL_DISTRO_NAME': 'Ubuntu',
        'VSCODE_INJECT_NODE_MODULE_LOOKUP_PATH': '/home/nlourenco/.vscode-server/bin/379476f0e13988d90fab105c5c19e7abc8b1dea8/remote/node_modules',
        'VSCODE_PIPE_LOGGING': 'true',
        'VSCODE_AMD_ENTRYPOINT': 'vs/server/remoteExtensionHostProcess',
        'NAME': 'Code',
        'PWD': '/mnt/c/Users/Nuno/AppData/Local/Programs/Microsoft VS Code',
        'LOGNAME': 'nlourenco',
        'MOTD_SHOWN': 'update-motd',
        'HOME': '/home/nlourenco',
        'LANG': 'C.UTF-8',
        'WSL_INTEROP': '/run/WSL/7_interop',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*